# Monte Carlo Justificaction

## MoSDeF Cassandra

In [4]:
import mbuild
import foyer
import mosdef_cassandra as mc
import unyt as u

# Use mbuild to create a methane
adsorbate = mbuild.load("Ar", smiles=True)

# Create an empty mbuild.Box
box = mbuild.Box(lengths=[10.0, 10.0, 10.0])

# Load force field
ff = foyer.Forcefield("../../Resources/foyer/noble_gas.xml")

# Use foyer to apply the force field
adsorbate_ff = ff.apply(adsorbate)

# Create box and species list
box_list = [box]
species_list = [adsorbate_ff]

mols_to_add = [[64]]

system = mc.System(box_list, species_list, mols_to_add=mols_to_add)
moveset = mc.MoveSet("gcmc", species_list)

custom_args = {
    "chemical_potentials": [-10.0 * (u.kJ / u.mol)],
    "prop_freq": 100,
}

mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=64000,
    temperature=100.0 * u.K,
    **custom_args,
)


################################################################

The code at conversion.py:252 requires the "rdkit" package

rdkit can be installed with conda using:

# conda install -c conda-forge rdkit

or from source following instructions at:

https://www.rdkit.org/docs/Install.html#installation-from-source

################################################################


DelayImportError: 
The code at conversion.py:252 requires the "rdkit" package

rdkit can be installed with conda using:

# conda install -c conda-forge rdkit

or from source following instructions at:

https://www.rdkit.org/docs/Install.html#installation-from-source


## ASE-LJ

## ASE-CP2K

# Analysis